In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_ollama.llms import OllamaLLM

model = OllamaLLM(model="gemma3:1b")

In [ ]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回覆客戶郵件。請用友善和專業的語調回覆。請不要回覆(公司的資訊和結尾詞),我會自已手動增加"),
    ("human","請回覆這封客戶郵件：\n\n{email_content}")
])

In [ ]:
def format_email_reply(ai_answer:str) -> str:
    import datetime
    date_str = datetime.datetime.now().strftime("中華民國,西元%Y-%m-%d %H:%M")
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
致理客服團隊

回覆時間:{date_str}
"""

    print(f"ai回覆的內容:{formatted_reply}")

    return {"formatted_reply":formatted_reply}

format_email = RunnableLambda(format_email_reply)

In [ ]:
improve_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
        ("human","估並改進這封郵件:\n\n{formatted_reply}")
    ]
)

In [ ]:
chain = (prompt_template 
        | model
        | StrOutputParser()
        | format_email
        | improve_prompt
        | model
        | StrOutputParser()
)

In [ ]:
customer_email = """您好:

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。


王小明"""

In [ ]:
result = chain.invoke({
    "email_content":customer_email
})

print(result)